In [106]:
import sys
import csv
from langdetect import detect
import nltk
from nltk.corpus import stopwords
import string
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import nltk
import pandas as pd
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()
import os.path
from os import path

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/francisbian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [161]:
print(string.punctuation)
stop_words = set(stopwords.words('english'))
stop_words.add('verse')
print(stop_words)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
{'again', 'up', 'your', 'd', 'doing', 'few', 'ourselves', 'it', 'aren', 'over', 'yourself', 'have', 'no', 'those', 'am', 'was', 'than', 'we', "you'll", 'can', 'into', 'but', 'then', 'do', 'ain', 'haven', 'its', 'won', 'such', 'verse', "shan't", 'this', 'them', 'while', "doesn't", 'is', 'mustn', 'so', 'shouldn', "it's", 'after', 'having', 'his', 'been', 'being', 'through', 'where', 'should', 'to', 'are', 'himself', "won't", 'about', 'just', 'each', "mightn't", 'me', "you've", 'of', "didn't", 'hasn', 'will', 'before', 'further', 'how', "hasn't", 'needn', 'now', 'wasn', "wouldn't", 'isn', "isn't", 'you', 'only', 'why', 'in', 'yours', 'more', 'most', 'with', 'both', 'wouldn', 'if', "aren't", 'same', "shouldn't", 'what', 'that', 'nor', 'a', 'hers', 'because', "should've", 'when', 'she', 'against', 'they', 'herself', 'these', 'did', 'for', 'all', "needn't", 'own', "she's", 'there', 's', 'has', 'shan', "haven't", 'and', 'didn', 'our', 'does', 'i', 'him', 'be',

## Here I take songs of 1960 as an example

### Let's combine the two lists based on hotlist_100 list's id. Set features to null if not found in spotify list
### I kept the title of songs from both lists. That is because the title from spotify seems to be more accurate, while the title from hotlist seems to be easier for searching

In [97]:
hotlist_csv = pd.read_csv('../dataset_hot_list/lyricsHotListTop_{}.csv'.format(1960))
spotify_csv = pd.read_csv('../spotify_features_csvs/lyricsHotListTop_{}.csv'.format(1960))
combined_csv = pd.merge(hotlist_csv, spotify_csv, how='left', on='Unnamed: 0')

In [98]:
combined_csv

,Unnamed: 0,title,lyrics,artists,date,rank,cnt,song_name,song_link,song_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_genres,album_genres
0,0,Are You Lonesome To-night?,[Verse 1]\nAre you lonesome tonight?\nDo you m...,Elvis Presley,1960,"[1, 1, 1, 1, 1, 2, 35]",7,Are You Lonesome To-night? (In the Style of El...,https://open.spotify.com/track/4oQ7A4Zq10kdSgI...,4oQ7A4Zq10kdSgIvlaIjbL,...,-7.557,1.0,0.0277,0.81800,0.769000,0.1740,0.2010,78.883,['karaoke'],[]
1,1,Wonderland By Night,Instrumental,Floyd Cramer,1960,"[2, 17, 27, 3, 20, 47, 4, 25, 57, 13, 51, 89, ...",18,Wonderland By Night,https://open.spotify.com/track/30WUWEzisQFY1df...,30WUWEzisQFY1dfgpVRXv3,...,-13.178,1.0,0.0326,0.86100,0.078900,0.1120,0.3210,107.429,"['easy listening', 'honky-tonk piano', 'nashvi...",[]
2,2,Last Date,It's over our love affair\nToo late now I find...,Floyd Cramer,1960,"[3, 41, 2, 24, 2, 21, 2, 39, 2, 70, 4, 94, 8, ...",19,Last Date - Instrumental,https://open.spotify.com/track/7rbvjRc6ZTVA3GU...,7rbvjRc6ZTVA3GUXDx3Xtr,...,-12.026,1.0,0.0289,0.85800,0.958000,0.1500,0.6130,103.818,"['easy listening', 'honky-tonk piano', 'nashvi...",[]
3,3,A Thousand Stars,ONEUS「A Thousand Stars (アサウザンドスターズ)」の歌詞\n\n「イン...,ONEUS,1960,"[4, 7, 3, 5, 5, 7, 9, 22, 37, 67]",10,A Thousand Stars,https://open.spotify.com/track/0295jaQ5AXCo4he...,0295jaQ5AXCo4he9qj22GB,...,-3.220,1.0,0.0959,0.19100,0.000000,0.3470,0.4850,135.061,"['k-pop', 'k-pop boy group']",[]
4,4,Exodus,"[Intro: YUNG PLAGUE]\nFirst, I turned the Nile...",$UICIDEBOY$,1960,"[5, 6, 10, 30, 40, 51, 71]",7,Exodus,https://open.spotify.com/track/2Bkfvi3pCcwoJwn...,2Bkfvi3pCcwoJwnmd45OnA,...,-4.497,1.0,0.0728,0.00641,0.000077,0.2340,0.1800,121.829,"['dark trap', 'new orleans rap', 'underground ...",[]
5,5,North To Alaska,Way up north (north to Alaska)\nWay up north (...,Johnny Horton,1960,"[6, 4, 5, 6, 7, 12, 12, 18, 23, 31, 37, 59, 64...",15,North To Alaska*,https://open.spotify.com/track/3JLJ0n5ZFd8oOwv...,3JLJ0n5ZFd8oOwvQv7LUEt,...,-14.436,1.0,0.0390,0.71800,0.000000,0.3480,0.9160,82.909,"['country', 'cowboy western', 'rock-and-roll']",[]
6,6,Many Tears Ago,A long time ago in a far off place\nI got caug...,Brett James,1960,"[7, 8, 8, 11, 14, 19, 49, 79]",8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,You're Sixteen,"[Verse]\nHo!\nYou come on like a dream, peache...",Ringo Starr,1960,"[8, 9, 12, 17, 21, 32, 41, 62, 94]",9,You're Sixteen (You’re Beautiful And You’re Mine),https://open.spotify.com/track/4MAxWahEbzU8m4b...,4MAxWahEbzU8m4bD6mSZg9,...,-12.852,1.0,0.0441,0.03810,0.000018,0.0637,0.8560,124.265,"['album rock', 'bubblegum pop', 'classic rock'...",[]
8,8,Sailor (Your Home Is The Sea),Ueber Rio und Shanghai\nUeber Bali und Hawaii\...,Lolita,1960,"[9, 5, 6, 9, 12, 24, 34, 65, 73, 76]",10,Sailor (Your Home Is The Sea),https://open.spotify.com/track/3l477StNmG8dGEw...,3l477StNmG8dGEwGzFshxh,...,-10.041,1.0,0.0325,0.82600,0.000040,0.3500,0.3880,102.537,[],[]
9,9,"Corinna, Corinna","[Songtext von ""Corinna, Corinna""]\n\nCorrina, ...",Wolfgang Ambros,1960,"[10, 27, 42, 71, 84, 96]",6,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### After we do extraction of the lyrics

In [156]:
# For now, I skip the songs whose lyrics is not English or length is smaller than 200 characters

def extract_lyrics(combined_csv):
    combined_csv.insert(3, 'words', np.empty((len(combined_csv), 0)).tolist())
    combined_csv.insert(4, 'lyrics_length', [0]*len(combined_csv))
    for i in range(len(combined_csv)):
        lyric = combined_csv.loc[i, 'lyrics']
        if pd.isnull(combined_csv.loc[i, 'lyrics']):
            continue
        if len(lyric) < 200 or detect(lyric) != 'en':
            continue
        li = []
        for r in lyric.splitlines():
            # Skip sth like '[Instrument]',['Bridge']
            if '[' in r or '(' in r: 
                continue
            for w in r.split():
                combined_csv.loc[i, 'lyrics_length'] += 1
                #if w in stop_words:
                #    continue;
                word = ''.join([ch for ch in w if ch not in string.punctuation]).lower()
                word = lem.lemmatize(word,"v")
                combined_csv.loc[i, 'words'].append(word)


In [100]:
extract_lyrics(combined_csv)

In [101]:
combined_csv

,Unnamed: 0,title,lyrics,words,lyrics_length,artists,date,rank,cnt,song_name,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_genres,album_genres
0,0,Are You Lonesome To-night?,[Verse 1]\nAre you lonesome tonight?\nDo you m...,"[be, you, lonesome, tonight, do, you, miss, me...",222,Elvis Presley,1960,"[1, 1, 1, 1, 1, 2, 35]",7,Are You Lonesome To-night? (In the Style of El...,...,-7.557,1.0,0.0277,0.81800,0.769000,0.1740,0.2010,78.883,['karaoke'],[]
1,1,Wonderland By Night,Instrumental,[],0,Floyd Cramer,1960,"[2, 17, 27, 3, 20, 47, 4, 25, 57, 13, 51, 89, ...",18,Wonderland By Night,...,-13.178,1.0,0.0326,0.86100,0.078900,0.1120,0.3210,107.429,"['easy listening', 'honky-tonk piano', 'nashvi...",[]
2,2,Last Date,It's over our love affair\nToo late now I find...,"[its, over, our, love, affair, too, late, now,...",138,Floyd Cramer,1960,"[3, 41, 2, 24, 2, 21, 2, 39, 2, 70, 4, 94, 8, ...",19,Last Date - Instrumental,...,-12.026,1.0,0.0289,0.85800,0.958000,0.1500,0.6130,103.818,"['easy listening', 'honky-tonk piano', 'nashvi...",[]
3,3,A Thousand Stars,ONEUS「A Thousand Stars (アサウザンドスターズ)」の歌詞\n\n「イン...,[],0,ONEUS,1960,"[4, 7, 3, 5, 5, 7, 9, 22, 37, 67]",10,A Thousand Stars,...,-3.220,1.0,0.0959,0.19100,0.000000,0.3470,0.4850,135.061,"['k-pop', 'k-pop boy group']",[]
4,4,Exodus,"[Intro: YUNG PLAGUE]\nFirst, I turned the Nile...","[first, i, turn, the, nile, river, into, blood...",332,$UICIDEBOY$,1960,"[5, 6, 10, 30, 40, 51, 71]",7,Exodus,...,-4.497,1.0,0.0728,0.00641,0.000077,0.2340,0.1800,121.829,"['dark trap', 'new orleans rap', 'underground ...",[]
5,5,North To Alaska,Way up north (north to Alaska)\nWay up north (...,"[north, to, alaska, , go, north, the, rush, be...",257,Johnny Horton,1960,"[6, 4, 5, 6, 7, 12, 12, 18, 23, 31, 37, 59, 64...",15,North To Alaska*,...,-14.436,1.0,0.0390,0.71800,0.000000,0.3480,0.9160,82.909,"['country', 'cowboy western', 'rock-and-roll']",[]
6,6,Many Tears Ago,A long time ago in a far off place\nI got caug...,"[a, long, time, ago, in, a, far, off, place, i...",180,Brett James,1960,"[7, 8, 8, 11, 14, 19, 49, 79]",8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,You're Sixteen,"[Verse]\nHo!\nYou come on like a dream, peache...","[ho, you, come, on, like, a, dream, peach, and...",195,Ringo Starr,1960,"[8, 9, 12, 17, 21, 32, 41, 62, 94]",9,You're Sixteen (You’re Beautiful And You’re Mine),...,-12.852,1.0,0.0441,0.03810,0.000018,0.0637,0.8560,124.265,"['album rock', 'bubblegum pop', 'classic rock'...",[]
8,8,Sailor (Your Home Is The Sea),Ueber Rio und Shanghai\nUeber Bali und Hawaii\...,"[ueber, rio, und, shanghai, ueber, bali, und, ...",42,Lolita,1960,"[9, 5, 6, 9, 12, 24, 34, 65, 73, 76]",10,Sailor (Your Home Is The Sea),...,-10.041,1.0,0.0325,0.82600,0.000040,0.3500,0.3880,102.537,[],[]
9,9,"Corinna, Corinna","[Songtext von ""Corinna, Corinna""]\n\nCorrina, ...",[],0,Wolfgang Ambros,1960,"[10, 27, 42, 71, 84, 96]",6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
combined_csv.to_csv('../combined_dataset/lyrics&features_{}.csv'.format(1960))

## Let's combine all the dataset we have and save to the directory

In [160]:
for year in range(1960, 2021):
    hotlist_csv = pd.read_csv('../dataset_hot_list/lyricsHotListTop_{}.csv'.format(year))
    combined_csv = hotlist_csv
    # Skip 
    if path.exists('../spotify_features_csvs/lyricsHotListTop_{}.csv'.format(year)):
        spotify_csv = pd.read_csv('../spotify_features_csvs/lyricsHotListTop_{}.csv'.format(year))
        combined_csv = pd.merge(hotlist_csv, spotify_csv, how='left', on='Unnamed: 0')
    
    extract_lyrics(combined_csv)
    combined_csv.to_csv('../combined_dataset/lyrics&features_{}.csv'.format(year)) 